In [3]:
import sys, os
sys.path.append("..")
from common_utils import DATA_HOME
from lstm.sales_data import Sales_Dataset
from torch.utils.data.dataloader import DataLoader
import torch
import random

torch.manual_seed(42)
random.seed(42)

In [4]:
SALE_HOME = os.path.join(DATA_HOME, "sales_data")

I, H, B = 71, 34, 3

sd = Sales_Dataset(SALE_HOME, seq_len=500)
test_set, train_set = torch.utils.data.random_split(sd, [0.2, 0.8])
train_dl = DataLoader(train_set, shuffle=True, batch_size=B)
test_dl = DataLoader(test_set, shuffle=True, batch_size=B)
len(train_dl), len(test_dl)

# # print(sd.O.describe())
# # print(sd.TR.describe())
# # print(sd.TS.describe())
# # print(sd.S.describe())
print(sd[4][0].shape)
# print(sd[4][1].shape)
# # print(sd[4][0], sd[4][1])


torch.Size([500, 71])


### PyTorch Model training

In [5]:
from torch.nn import LSTM
from torch.nn import MSELoss
from torch import optim
import matplotlib.pyplot as plt

model = LSTM(I, H, batch_first=True).cuda()
loss = MSELoss()
adam = optim.Adam(model.parameters(), lr=0.001)
h, c = torch.randn(1, B, H, dtype=torch.float32).cuda(), torch.randn(1, B, H, dtype=torch.float32).cuda()
tr_losses, inf_losses = [], []

train_gen = enumerate(train_dl)
test_gen = enumerate(test_dl)

for tr_idx, (X, y) in train_gen:
    tr_l, inf_l = 0, 0
    yhat, (_, _) = model(X)
    l = loss(yhat, y.cuda())

    adam.zero_grad()
    l.backward()
    adam.step()
    tr_l += l.item()

    try:
        inf_idx, (X, y) = next(test_gen)
    except StopIteration:
        test = enumerate(test_dl)
        inf_idx, (X, y) = next(test_gen)
    yhat, (_, _) = model(X)
    l = loss(yhat, y.cuda())
    inf_l += l.item()
    tr_losses.append(tr_l / len(train_dl))
    inf_losses.append(inf_l / len(test_dl))
    if tr_idx and tr_idx % 500 == 0:
        print(f"iteration: {tr_idx} train loss: {tr_l / len(train_dl)} inf loss: {inf_l / len(test_dl)}")
        torch.save(model.state_dict(), f"old_sales_model_{tr_idx}.pth")


iteration: 500 train loss: 3.693451010770436e-06 inf loss: 1.2484052267926359e-05
iteration: 1000 train loss: 3.708210625305372e-06 inf loss: 1.302627232104282e-05
iteration: 1500 train loss: 2.0972811660782213e-06 inf loss: 1.6562933324791992e-05
iteration: 2000 train loss: 2.5599164035825216e-06 inf loss: 1.6265286375268308e-05


KeyboardInterrupt: 

### display losses

In [ ]:
plt.plot(range(len(tr_losses)), tr_losses, label="train loss")
plt.plot(range(len(inf_losses)), inf_losses, label="test loss")
plt.legend()
plt.show()
